<h3>Run the cell below to initialize the program. Remember to fill in the base_dir, project_name, raw_data_file_name and chunk_nums fields</h3>

In [ ]:
import os
import sys
%run ..//NewFunctions.py


#****************** fill these in *************************
base_dir = 'D:\\Segment'
project_name = 'K02288'
raw_data_file_name = 'Decon_'

#ignore this if only working with a single part 
chunk_nums = [1, 1]
#****************** fill these in *************************

path = os.path.join(base_dir, project_name)

Path(os.path.join(path, 'processed_images', 'mem')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(path, 'processed_images', 'nuc')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(path, 'processed_images', 'nuc', 'temp')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(path, 'processed_images', 'mem', 'temp')).mkdir(parents=True, exist_ok=True)

total_file_nums = len(os.listdir(os.path.join(path, 'raw_data', 'nuc')))   

%gui qt

<h3>Align multiple parts of a timelapse sequence (if necessary)</h3>

In [ ]:
#if there are multiple parts, align and rename them all

last_part_1 = io.imread(path + '\\raw_data\\mem\\part_1\\' + raw_data_file_name + str(chunk_nums[0]-1) + '.tif')
first_part_2 = io.imread(path + '\\raw_data\\mem\\part_2\\' + raw_data_file_name + str(0) + '.tif')

combined_stacks = np.stack((last_part_1, first_part_2), axis=0)

viewer = DDN.PreProcess.OpenAlignmentViewer(combined_stacks, path, chunk_nums)

#rename all files and put them into a single folder, under the mem or nuc folders (where the 'normal' would otherwise be)

<h3>Downsample the images by a factor of two</h3>

In [ ]:
#downsample the images by a factor of 2:
files = []
for entry in os.scandir(os.path.join(path, 'processed_images', 'nuc')):
    if entry.is_file():
        files.append(entry)
total_file_nums = len(files)


for i in range (0, total_file_nums):
    
    clear_output(wait=True)
    print('running file ' + str(i) + ' of ' + str(total_file_nums-1))
    
    im = io.imread(os.path.join(path, 'raw_data', 'mem', raw_data_file_name + str(i) + '.tif'))                            
    im_zoomed = zoom(im, (0.5, 0.5, 0.5), order = 1)    
    np.savez_compressed(os.path.join(path, 'processed_images', 'mem', 'temp', 'mem_' + str(i)), im_zoomed)
    
    im = io.imread(os.path.join(path, 'raw_data', 'nuc', raw_data_file_name + str(i) + '.tif'))
    im_zoomed = zoom(im, (0.5, 0.5, 0.5), order = 1)
    np.savez_compressed(os.path.join(path, 'processed_images', 'nuc', 'temp', 'nuc_' + str(i)), im_zoomed)
        

<h3> Crop the images by defning control points to crop to in X, Y and Z planes </h3>

In [ ]:
DDN.PreProcess.OpenCropViewer(path, total_file_nums-1)

<h3>Align Red and Green channels (if necessary) </h3>

In [ ]:
DDN.PreProcess.AlignRG(path, total_file_nums-1)

<h3>Rotate the images (if necessary) </h3>

In [ ]:
DDN.PreProcess.RotateImages(path, total_file_nums-1)

<h3> Check the data and finalize the processed images </h3>

In [ ]:
DDN.PreProcess.CheckAllFiles(path, start=0, end=total_file_nums-1)